In [1]:
import numpy as np

In [2]:
def pad_img(img: np.ndarray, padding: int) -> np.ndarray:
    # Get size of image
    num_rows = img.shape[0]
    num_cols = img.shape[1]
    
    # Create padding for rows
    zero_rows = np.zeros((padding, num_cols))
    # Add padding to each side of the image
    img = np.vstack((zero_rows, img))
    img = np.vstack((img, zero_rows))
    
    # Create padding for columns (need to add 2*padding
    # as the image is now wider)
    zero_cols = np.zeros((num_rows+2*padding, padding))
    # Add padding to top and bottom of the image
    img = np.hstack((zero_cols, img))
    img = np.hstack((img, zero_cols))
    
    return img

In [3]:
def convolute_2d(img: np.ndarray, kernel: np.ndarray, padding: int) -> np.ndarray:
    # Assume square mask
    kernel_size = kernel.shape[0]
    img_pad = pad_img(img, padding)
    num_rows = img_pad.shape[0]
    num_cols = img_pad.shape[1]
    # Want to make sure that the output is the size of
    # the image without padding
    img_conv = np.zeros((num_rows-2*padding, num_cols-2*padding))
    
    # The +1 is needed to get the center to be the first pixel
    # The padding gives half of it but as it is odd you need to add 1
    for i in range(num_rows - kernel_size + 1):
        for j in range(num_cols - kernel_size + 1):
            img_window = img_pad[i:i+kernel_size, j:j+kernel_size]
            img_conv[i, j] = (img_window.flatten()*kernel.flatten()).sum()
    return img_conv

In [4]:
def activation_relu(img: np.ndarray) -> np.ndarray:
    # ReLu leaves positives the same, but sets negatives to 0
    img[img<0] = 0
    return img

In [5]:
def pool_convolution(img: np.ndarray) -> np.ndarray:
    num_rows = img.shape[0]
    num_cols=img.shape[1]
    max_pool = np.zeros((int(num_rows/2), int(num_cols/2)))
    # Shift the window across in steps of 2
    for i in range(0, num_rows, 2):
        for j in range(0, num_cols, 2):
            # Get a window of the image
            img_window = img[i:i+2, j:j+2]
            # Take the maximum value of the window and
            # set it as the value in teh max pool
            max_pool[int(i/2), int(j/2)] = img_window.max()
    return max_pool
            

In [6]:
def activation_sigmoid(flattened: np.ndarray,
                       weights: np.ndarray,
                       bias: float) -> float:
    # Get the activation value
    x = weights.dot(flattened) + bias
    # This is just the sigmoid function
    y_hat = 1/(1+np.exp(-x))
    return y_hat

In [7]:
def forward_pass(img: np.ndarray,
                 kernel: np.ndarray,
                 bias_in: float,
                 weights: np.ndarray,
                 bias_out:float) -> float:
    # Get padding size
    padding = int(kernel.shape[0]/2)
    # Convolute image
    img_conv = convolute_2d(img, kernel, padding)
    # Add bias term and ReLu activation
    img_conv = img_conv + bias_in
    img_conv = activation_relu(img_conv)
    # Max pool the image to shrink it, then flatten
    img_pool = pool_convolution(img_conv)
    flattened_pool = img_pool.flatten()
    # Get y_hat
    y_hat = activation_sigmoid(flattened_pool, weights, bias_out)
    return y_hat